# MHA Transformer 

## Introduction
Welcome to this Jupyter notebook developed for Stanford Ribonanza RNA Folding to create a model that predicts the structures of any RNA molecule using sequence to sequence NLP techniques.

## Purpose
The primary purpose of this notebook is to:
- Load and preprocess the competition data 📁
- Engineer relevant features for model training 🏋️‍♂️
- Train predictive models to make target variable predictions 🧠
- Submit predictions to the competition environment 📤


## 📦 Import necessary libraries and modules


In [1]:
import torch  # 🔥 Import PyTorch for deep learning
from torch.utils.data import random_split  # 📂 For splitting datasets
from torch.utils.data import Dataset, DataLoader
import pandas as pd  # 🐼 Pandas for data manipulation
from pathlib import Path  # 📁 pathlib for handling file paths
import numpy as np  # 🧮 NumPy for numerical operations
from sklearn.preprocessing import OneHotEncoder  # 🧬 Scikit-Learn for one-hot encoding
import polars as pl  # 📊 Polars for data manipulation
import re  # 🧵 Regular expressions for text processing
from tqdm import tqdm  # 🔄 tqdm for progress bar display
import torch.nn as nn


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 📁 Define file paths for dataset and output


In [2]:
DATA_DIR = Path("/kaggle/input/stanford-ribonanza-rna-folding/")  # 📂 Directory for dataset
TRAIN_CSV = DATA_DIR / "train_data.csv"  # 🚆 Training data in CSV format
TRAIN_PARQUET_FILE = "train_data.parquet"  # 📦 Training data in Parquet format
TEST_CSV = DATA_DIR / "test_sequences.csv"  # 🚀 Test sequences in CSV format
TEST_PARQUET_FILE = "test_sequences.parquet"  # 📦 Test sequences in Parquet format
PRED_CSV = "submission.csv"  # 📄 Output file for predictions


In [3]:
def to_parquet(csv_file, parquet_file):
    # 📊 Read CSV data using Polars
    dummy_df = pl.scan_csv(csv_file)

    # 🔍 Define a new schema mapping for specific columns
    new_schema = {}
    for key, value in dummy_df.schema.items():
        if key.startswith("reactivity"):
            new_schema[key] = pl.Float32  # 📊 Convert 'reactivity' columns to Float32
        else:
            new_schema[key] = value

    # 📊 Read CSV data with the new schema and write to Parquet
    df = pl.scan_csv(csv_file, schema=new_schema)
    
    # 💾 Write data to Parquet format with specified settings
    df.sink_parquet(
        parquet_file,
        compression='uncompressed',  # No compression for easy access
        row_group_size=10,  # Adjust row group size as needed
    )


In [4]:
to_parquet(TRAIN_CSV, TRAIN_PARQUET_FILE)  # 🚆 Training data
to_parquet(TEST_CSV, TEST_PARQUET_FILE)    # 🚀 Test data


In [5]:
from torch.utils.data import Dataset, DataLoader
class Indexer(object):
    """
    Bijection between objects and integers starting at 0. Useful for mapping
    labels, features, etc. into coordinates of a vector space.

    Attributes:
        objs_to_ints
        ints_to_objs
    """
    def __init__(self):
        self.objs_to_ints = {}
        self.ints_to_objs = {}

    def __repr__(self):
        return str([str(self.get_object(i)) for i in range(0, len(self))])

    def __str__(self):
        return self.__repr__()

    def __len__(self):
        return len(self.objs_to_ints)

    def get_object(self, index):
        if (index not in self.ints_to_objs):
            return None
        else:
            return self.ints_to_objs[index]

    def contains(self, object):
        return self.index_of(object) != -1

    def index_of(self, object):
        if (object not in self.objs_to_ints):
            return -1
        else:
            return self.objs_to_ints[object]

    def add_and_get_index(self, object, add=True):
        if not add:
            return self.index_of(object)
        if (object not in self.objs_to_ints):
            new_idx = len(self.objs_to_ints)
            self.objs_to_ints[object] = new_idx
            self.ints_to_objs[new_idx] = object
        return self.objs_to_ints[object]


class SequenceDataset(Dataset):
    def __init__(self, parquet_name, sequence_index, context_length=50):
            
        self.parquet_name = parquet_name
        self.seq_indexer = sequence_index
        self.df = pl.read_parquet(self.parquet_name)
        self.df = self.df.filter(pl.col("SN_filter") == 1.0)
        # 🧬 Get reactivity column names using regular expression
        reactivity_match = re.compile('(reactivity_[0-9])')
        reactivity_names = [col for col in self.df.columns if reactivity_match.match(col)]
        # 📊 Select only the reactivity columns
        self.reactivity_df = self.df.select(reactivity_names)
        # 📊 Select the 'sequence' column
        self.sequence_df = self.df.select("sequence")
        self.context_length = context_length
        
    def __len__(self):
        """
        Your code here
        """
        return len(self.df)

    def parse_row(self, idx):
        sequence_row = self.sequence_df.row(idx)
        reactivity_row = self.reactivity_df.row(idx)
        # 🧬 Get the sequence string
        sequence = list(sequence_row[0])
        # 🧬 Encode the sequence array using the one-hot encoder
        sequence_indices = torch.LongTensor(np.array([self.seq_indexer.index_of(s) for s in sequence]))
        sequence_length = len(sequence)
        start_idx = random.randint(0, len(sequence) - context_length)
        selected_sequence = sequence_indices[start_idx : start_idx + context_length]
        reactivity = torch.LongTensor(np.array(reactivity_row, dtype=np.float32)[start_idx : start_idx + context_length])
        valid_mask = np.argwhere(~np.isnan(reactivity)).reshape(-1)
        torch_valid_mask = torch.tensor(valid_mask, dtype=torch.long)
        # 🧬 Replace nan values for reactivity with 0.0 TODO figure out how to smoth these, perhaps with information from other columns
        reactivity = np.nan_to_num(reactivity, copy=False, nan=0.0)
        
        return selected_sequence, reactivity, valid_mask
    
    def __getitem__(self, idx):
        # 📊 Get and parse data for the specified index
        data = self.parse_row(idx)
        return data

    

## 📚 Create the full training dataset and split it into training and validation datasets

In [6]:
vocab = ['A', 'G', 'U', 'C']
vocab_index = Indexer()
for char in vocab:
    vocab_index.add_and_get_index(char)
print(repr(vocab_index))

full_train_dataset = SequenceDataset(parquet_name=TRAIN_PARQUET_FILE, sequence_index=vocab_index, context_length=80)  # 🚆 Full training dataset
generator1 = torch.Generator().manual_seed(42)  # 🌱 Initialize random seed generator
train_dataset, val_dataset = random_split(full_train_dataset, [0.7, 0.3], generator1)  # 🎯 Split dataset into training (70%) and validation (30%)


['A', 'G', 'U', 'C']


## 🚂 Create data loaders for training and validation


In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=0)  # 📦 Training data loader
val_dataloader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=0)  # 📦 Validation data loader
print("Done")

Done


## 📉 Define loss functions for training and evaluation


In [8]:
# 📉 Define loss functions for training and evaluation
import torch.nn.functional as F

def loss_fn(output, target):
    # 🪟 Clip the target values to be within the range [0, 1]
    clipped_target = torch.clip(target, min=0, max=1)
    # 📉 Calculate the mean squared error loss
    mses = F.mse_loss(output, clipped_target, reduction='mean')
    return mses

def mae_fn(output, target):
    # 🪟 Clip the target values to be within the range [0, 1]
    clipped_target = torch.clip(target, min=0, max=1)
    # 📉 Calculate the mean absolute error loss
    maes = F.l1_loss(output, clipped_target, reduction='mean')
    return maes


## 🧠 Create a neural network model using Multi Head Attention

In [9]:
class FeedFoward(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model),
        )

    def forward(self, x):
        return self.net(x)


class Head(nn.Module):
    def __init__(self, seq_length, d_model, num_heads, d_internal):
        super().__init__()
        self.K = nn.Linear(d_model, d_internal)
        self.Q = nn.Linear(d_model, d_internal)
        self.V = nn.Linear(d_model, d_internal)
        self.w0 = nn.Linear(d_internal, d_model // num_heads)
        self.register_buffer('tril', torch.tril(torch.ones(seq_length, seq_length)))

    def forward(self, input_vecs):
        keys = self.K(input_vecs) # B, L, d_internal
        d_k = keys.shape[-1]
        queries = self.Q(input_vecs) # B, L, d_internal
        value = self.V(input_vecs) # B, L, d_internal
        weights = torch.matmul(queries, keys.transpose(-2, -1)) * d_k**-0.5# L, L
        weights = weights.masked_fill(self.tril == 0, float('-inf'))
        attention = torch.softmax(weights, dim=-1)

        logit = torch.matmul(attention , value) # B, L, d_internal
        logit = self.w0(logit)
        return logit

class MultiHeadAttention(nn.Module):

    def __init__(self, seq_length, d_model, num_heads, d_internal):
        super().__init__()
        self.heads = nn.ModuleList([Head(seq_length, d_model, num_heads, d_internal) for _ in range(num_heads)])
        self.linear1 = nn.Linear(d_model, d_model)

    def forward(self, input_vecs):
        out = torch.cat([head(input_vecs) for head in self.heads], dim=-1)
        out = self.linear1(out)
        return out

class MHATransformerLayer(nn.Module):
    def __init__(self, seq_length, d_model, num_heads, d_internal):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention( seq_length, d_model, num_heads, d_internal)
        self.ffwd = FeedFoward(d_model)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)

    def forward(self, input_vecs):
        x = self.multi_head_attention(self.ln1(input_vecs))
        x += input_vecs
        x = x + self.ffwd(self.ln2(x))

        return x

class MHATransformer(nn.Module):
    def __init__(self, vocab_size, num_positions, d_model, d_internal, num_layers, num_heads):
    
        super().__init__()
        self.num_positions = num_positions
        self.L = []
        for ly in range(num_layers):
            self.L.append(MHATransformerLayer(num_positions, d_model, num_heads, d_internal))
        self.transformer_layers = nn.Sequential(*self.L)
        self.classifier = nn.Linear(d_model, 1)
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.layer_norm = nn.LayerNorm(d_model)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, indices, batched=False):
        logit = self.embedding(indices)
        logit = self.transformer_layers(logit)
        logit = self.classifier(logit)
        logit = self.softmax(logit)
        if batched:
            return logit
        else:
            return logit.squeeze(0)

In [10]:
# 🛠️ Set the device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
context_length = 80
model = MHATransformer(vocab_size=4, num_positions=context_length, d_model=64, d_internal=32, num_layers=1, num_heads=8).to(device)


In [11]:
import random
# Make sure we are using the GPU
device
#print(len(train_dataloader))
#for  batch_x, batch_y, valid_mask in (pbar := tqdm(train_dataloader, position=0, leave=True)):
#    print(batch_x.shape)
#    print(batch_y.shape)

device(type='cuda')

In [12]:
n_epochs = 0

# 📈 Define the optimizer with learning rate and weight decay
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)

# 🚂 Iterate over epochs
for epoch in range(n_epochs):
    train_losses = []
    train_maes = []
    model.train()
    
    # 🚞 Iterate over batches in the training dataloader
    for  batch_x, batch_y, valid_mask in (pbar := tqdm(train_dataloader, position=0, leave=True)):
        batch_x, batch_y, valid_mask = batch_x.to(device), batch_y.to(device), valid_mask.to(device)
        optimizer.zero_grad()
        out = model(batch_x, batched=True)
        out = torch.squeeze(out)
        loss = loss_fn(out[valid_mask], batch_y[valid_mask].float())
        mae = mae_fn(out[valid_mask], batch_y[valid_mask])
        loss.backward()
        train_losses.append(loss.detach().cpu().numpy())
        train_maes.append(mae.detach().cpu().numpy())
        optimizer.step()
        pbar.set_description(f"Train loss {loss.detach().cpu().numpy():.4f}")
    
    # 📊 Print average training loss and MAE for the epoch
    print(f"Epoch {epoch} train loss: ", np.mean(train_losses))
    print(f"Epoch {epoch} train mae: ", np.mean(train_maes))
    
    val_losses = []
    val_maes = []
    model.eval()
    
    # 🚞 Iterate over batches in the validation dataloader
    for batch_x, batch_y, valid_mask in (pbar := tqdm(val_dataloader, position=0, leave=True)):
        batch_x, batch_y, valid_mask = batch_x.to(device), batch_y.to(device), valid_mask.to(device)
        out = model(batch_x, batched=True)
        out = torch.squeeze(out)
        loss = loss_fn(out[valid_mask], batch_y[valid_mask].float())
        mae = mae_fn(out[valid_mask], batch_y[valid_mask])
        val_losses.append(loss.detach().cpu().numpy())
        val_maes.append(mae.detach().cpu().numpy())
        pbar.set_description(f"Validation loss {loss.detach().cpu().numpy():.4f}")
    
    # 📊 Print average validation loss and MAE for the epoch
    print(f"Epoch {epoch} val loss: ", np.mean(val_losses))
    print(f"Epoch {epoch} val mae: ", np.mean(val_maes))


## 🗑️ Clear the GPU memory cache


In [13]:
torch.cuda.empty_cache()


## 📚 Define a custom dataset class for inference on a graph


**Explaination**: 



In [14]:
class InferenceGraphDataset(Dataset):
    def __init__(self, parquet_name, edge_distance=2, root=None, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        # 📄 Set the Parquet file name
        self.parquet_name = parquet_name
        # 📏 Set the edge distance for generating the adjacency matrix
        self.edge_distance = edge_distance
        # 🧮 Initialize the one-hot encoder for node features
        self.node_encoder = OneHotEncoder(sparse_output=False, max_categories=4)
        # 🧮 Fit the one-hot encoder to possible values (A, G, U, C)
        self.node_encoder.fit(np.array(['A', 'G', 'U', 'C']).reshape(-1, 1))
        # 📊 Load the Parquet dataframe
        self.df = pl.read_parquet(self.parquet_name)
        # 📊 Select the 'sequence' and 'id_min' columns
        self.sequence_df = self.df.select("sequence")
        self.id_min_df = self.df.select("id_min")

    def parse_row(self, idx):
        # 📊 Read the row at the given index
        sequence_row = self.sequence_df.row(idx)
        id_min = self.id_min_df.row(idx)[0]

        # 🧬 Get the sequence string and convert it to an array
        sequence = np.array(list(sequence_row[0])).reshape(-1, 1)
        # 🧬 Encode the sequence array using the one-hot encoder
        encoded_sequence = self.node_encoder.transform(sequence)
        # 📏 Get the sequence length
        sequence_length = len(sequence)
        # 📊 Get the edge index using nearest adjacency function
        edges_np = nearest_adjacency(sequence_length, n=self.edge_distance, loops=False)
        # 📏 Convert the edge index to a torch tensor
        edge_index = torch.tensor(edges_np, dtype=torch.long)

        # 📊 Define node features as the one-hot encoded sequence
        node_features = torch.Tensor(encoded_sequence)
        ids = torch.arange(id_min, id_min+sequence_length, 1)

        data = Data(x=node_features, edge_index=edge_index, ids=ids)

        return data

    def len(self):
        # 📏 Return the length of the dataset
        return len(self.df)

    def get(self, idx):
        # 📊 Get and parse data for the specified index
        data = self.parse_row(idx)
        return data


## 📚 Create an inference dataset and dataloader

In [15]:
#infer_dataset = InferenceGraphDataset(parquet_name=TEST_PARQUET_FILE, edge_distance=EDGE_DISTANCE)  # 🚀 Inference dataset
#infer_dataloader = DataLoader(infer_dataset, batch_size=128, shuffle=False, num_workers=2)  # 📦 Inference dataloader


## 🏭 Set the device to GPU if available, otherwise use CPU


In [16]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 🏭 Move the model to the selected device (GPU or CPU) and switch to evaluation mode
#model = model.eval().to(device)


## 🧮 Initialize empty arrays for IDs and predictions & Append IDs and predictions


In [17]:
#ids = np.empty(shape=(0, 1), dtype=int)
#preds = np.empty(shape=(0, 1), dtype=np.float32)

# 🚀 Iterate over batches in the inference dataloader
#for batch in tqdm(infer_dataloader):
#    batch = batch.to(device)
#    out = model(batch.x, batch.edge_index).detach().cpu().numpy()

    # 📦 Append IDs and predictions to the respective arrays
#    ids = np.append(ids, batch.ids.detach().cpu().numpy())
#    preds = np.append(preds, out)


## 📊 Create a DataFrame for the submission


In [18]:
#submission_df = pl.DataFrame({"id": ids, "reactivity_DMS_MaP": preds, "reactivity_2A3_MaP": preds})


## 💾 Write the submission DataFrame to a CSV file


In [19]:
#submission_df.write_csv(PRED_CSV)


In [20]:
class SimpleGraphDataset(Dataset):
    def __init__(self, parquet_name, edge_distance=5, root=None, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        # 📄 Set the Parquet file name
        self.parquet_name = parquet_name
        # 📏 Set the edge distance for generating the adjacency matrix
        self.edge_distance = edge_distance
        # 🧮 Initialize the one-hot encoder for node features
        self.node_encoder = OneHotEncoder(sparse_output=False, max_categories=5)
        # 🧮 Fit the one-hot encoder to possible values (A, G, U, C)
        self.node_encoder.fit(np.array(['A', 'G', 'U', 'C']).reshape(-1, 1))
        # 📊 Load the Parquet dataframe
        self.df = pl.read_parquet(self.parquet_name)
        # 📊 Filter the dataframe by 'SN_filter' column where the value is 1.0
        self.df = self.df.filter(pl.col("SN_filter") == 1.0)
        # 🧬 Get reactivity column names using regular expression
        reactivity_match = re.compile('(reactivity_[0-9])')
        reactivity_names = [col for col in self.df.columns if reactivity_match.match(col)]
        # 📊 Select only the reactivity columns
        self.reactivity_df = self.df.select(reactivity_names)
        # 📊 Select the 'sequence' column
        self.sequence_df = self.df.select("sequence")
        

    def parse_row(self, idx):
        # 📊 Read the row at the given index
        sequence_row = self.sequence_df.row(idx)
        reactivity_row = self.reactivity_df.row(idx)
        # 🧬 Get the sequence string and convert it to an array
        sequence = np.array(list(sequence_row[0])).reshape(-1, 1)
        # 🧬 Encode the sequence array using the one-hot encoder
        encoded_sequence = self.node_encoder.transform(sequence)
        # 📏 Get the sequence length
        sequence_length = len(sequence)
        # 📊 Get the edge index using nearest adjacency function
        edges_np = nearest_adjacency(sequence_length, n=self.edge_distance, loops=False)
        # 📏 Convert the edge index to a torch tensor
        edge_index = torch.tensor(edges_np, dtype=torch.long)
        # 🧬 Get reactivity targets for nodes
        reactivity = np.array(reactivity_row, dtype=np.float32)[0:sequence_length]
        # 🔒 Create valid masks for nodes
        valid_mask = np.argwhere(~np.isnan(reactivity)).reshape(-1)
        torch_valid_mask = torch.tensor(valid_mask, dtype=torch.long)
        # 🧬 Replace nan values for reactivity with 0.0 (not super important as they get masked)
        reactivity = np.nan_to_num(reactivity, copy=False, nan=0.0)
        # 📊 Define node features as the one-hot encoded sequence
        node_features = torch.Tensor(encoded_sequence)
        # 🎯 Define targets
        targets = torch.Tensor(reactivity)
        # 📊 Create a PyTorch Data object
        data = Data(x=node_features, edge_index=edge_index, y=targets, valid_mask=torch_valid_mask)
        return data

    def len(self):
        # 📏 Return the length of the dataset
        return len(self.df)

    def get(self, idx):
        # 📊 Get and parse data for the specified index
        data = self.parse_row(idx)
        return data